# Прогноз судьбы стартапов с использованием CatBoost

## Описание проекта

Цели проекта:
* разработать ML-модель для прогнозирования успешности / неуспешности стартапа;
* подготовить рекомендации будущим стартаперам.

Исходные данные: база данных о стартапах с 1980 по 2018 годы.

Целевая метрика: F1 (среднее гармоническое precision и recall).

## Импорт библиотек и настройки

In [ ]:
!pip install --upgrade matplotlib
!pip install --upgrade seaborn
!pip install optuna-integration[catboost]

Импортируем необходимые библиотеки.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import phik
import shap
import numpy as np
import warnings

from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from catboost import CatBoostClassifier, Pool
import optuna
from optuna.integration import CatBoostPruningCallback
from optuna.samplers import TPESampler

Установим значение для генератора случайных чисел.

In [ ]:
RANDOM_STATE = 42

Снимем ограничение на максимальное число столбцов в выводе.

In [ ]:
pd.options.display.max_columns = None

Выключим предупреждения.

In [ ]:
warnings.filterwarnings('ignore')

## Общий обзор данных

### Загрузка данных

Загрузим исходные датасеты.

In [ ]:
try:
    startups_train = pd.read_csv('kaggle_startups_train_28062024.csv')
    startups_test = pd.read_csv('kaggle_startups_test_28062024.csv')
except:
    startups_train = pd.read_csv('/kaggle/input/startups-operations-close-predictions-m-1-39-ds/kaggle_startups_train_28062024.csv')
    startups_test = pd.read_csv('/kaggle/input/startups-operations-close-predictions-m-1-39-ds/kaggle_startups_test_28062024.csv')

### Обзор данных

Рассмотрим датасет startups_train.

In [ ]:
startups_train.info()
startups_train.head()

Видны пропуски в ряде столбцов.

Рассмотрим датасет startups_test.

In [ ]:
startups_test.info()
startups_test.head()

##  Предобработка данных

Параллельно будем создавать пайплайн.

In [ ]:
startups_train_proc = startups_train.copy()
startups_test_proc = startups_test.copy()

### Изменение типов данных

Создадим функцию (для использования в пайплайне в дальнейшем)

In [ ]:
def to_date(X) -> pd.DataFrame :
    return X.apply(pd.to_datetime)

In [ ]:
date_features = ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at']
startups_train_proc[date_features] = to_date(startups_train_proc[date_features])
startups_train_proc.info()

In [ ]:
date_features = ['first_funding_at', 'last_funding_at']
startups_test_proc[date_features] = to_date(startups_test_proc[date_features])
startups_test_proc.info()

### Обработка пропусков

Посчитаем число пропусков.

Предварительно выделим категориальные признаки (пригодится в будущем для CatBoost)

In [ ]:
for column in startups_train_proc:
    print(f'Число пропусков в {column}: {startups_train_proc[column].isna().sum()}')

Проверим пропуск в поле name.

In [ ]:
startups_train_proc.query('name.isna()')

Поскольку остальные признаки не пропущены, оставим этот пропуск как есть (тем более, что он не понадобится для модели)

In [ ]:
for column in startups_test_proc:
    print(f'Число пропусков в {column}: {startups_test_proc[column].isna().sum()}')

Заполним пропуски в категориальных столбцах.

In [ ]:
cat_features = ['category_list', 'country_code', 'state_code', 'region', 'city']

In [ ]:
cat_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='N/A')

In [ ]:
startups_train_proc[cat_features] = cat_imputer.fit_transform(startups_train_proc[cat_features])
startups_train_proc[cat_features].isna().sum()

In [ ]:
startups_test_proc[cat_features] = cat_imputer.transform(startups_test_proc[cat_features])
startups_test_proc[cat_features].isna().sum()

### Обработка дубликатов

Подсчитаем число полных дубликатов.

In [ ]:
startups_train_proc.duplicated().sum()

In [ ]:
startups_test_proc.duplicated().sum()

Подсчитаем число дубликатов по полю name.

In [ ]:
startups_train_proc['name'].duplicated().sum()

In [ ]:
startups_test_proc['name'].duplicated().sum()

Для поиска неявных дубликатов рассмотрим уникальные значения категориальных признаков.

In [ ]:
for column in cat_features:
    for name, df in {'тренировочной выборке': startups_train_proc, 'тестовой выборке': startups_test_proc}.items():
        print(f'Уникальные значения в "{column}" в {name}:', df[column].unique())
        print(f'Число уникальных значений в "{column}":', df[column].nunique())
        print()

### Добавление в тренировочную выборку признака продолжительности жизни стартапа

Добавление нового признака в тренировочной выборке - продолжительности жизни стартапа - аналогично тестовой выборке.

In [ ]:
def compute_lifetime(X, name='lifetime') -> pd.DataFrame:
    X = X.copy()
    if X.shape[1] != 2:
        raise Exception('Should be only 2 columns')
    X.iloc[:, 0] = X.iloc[:, 0].fillna(datetime(2018, 1, 1))
    X[name] = pd.Series(X.iloc[:, 0] - X.iloc[:, 1]).dt.days
    return X

In [ ]:
lifetimer = ColumnTransformer(
    [
        ('lifetimer', FunctionTransformer(compute_lifetime), ['closed_at', 'founded_at'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
startups_train_proc  = lifetimer.fit_transform(startups_train_proc)
startups_train_proc.info()

### Логические проверки

Посмотрим, как соотносятся различные даты.

Сначала сравним даты основания и даты первого финансирования.

In [ ]:
len(startups_train_proc.query('founded_at > first_funding_at'))

Более 2900 стартапов имеют дату основания позже, чем дата первого финансирования. Однако такое возможно: финансирование на посевной стадии может происходить до официального основания стартапа (многие стартапы рассматривают в качестве даты основания дату выпуска первого продукта).

Сравним даты основания и закрытия.

In [ ]:
len(startups_train_proc.query('founded_at > closed_at'))

Сначала сравним даты первого и последнего финансирования.

In [ ]:
len(startups_train_proc.query('first_funding_at > last_funding_at'))

Сначала сравним даты последнего финансирования и закрытия.

In [ ]:
len(startups_train_proc.query('last_funding_at > closed_at'))

##  Исследовательский анализ данных

### Описательная статистика и графики числовых параметров

Определим функции для построения графиков параметров.

In [ ]:
# Функция для построения графиков числовых непрерывных параметров.
# Функция строит "сетку" графиков для различных столбцов датафрейма.
# Параметры функции:
# df - датафрейм с исходными данными;
# columns - список столбцов, по которым строятся графики;
# num_cols - число столбцов "сетке" графиков;
# title - общий заголовок для "сетки";
# hue - столбец, для уникальных значений которого строятся дополнительные графики

def plot_quant_interval(df, columns, num_cols, title, hue=None):
    num_rows = -(-len(columns) // num_cols) # округление вверх

    fig = plt.figure(figsize=(num_cols*5, num_rows*3))
    outer_grid = fig.add_gridspec(num_rows, num_cols, hspace=0.5, wspace=0.2)

    i = 0

    color = '0.8' if hue else 'C0'

    for column in columns:
        inner_grid = outer_grid[i].subgridspec(2, 1, height_ratios=[2, 1],
                                               hspace=0, wspace=0)
        ax = inner_grid.subplots(sharex=True)
        sns.histplot(data=df, x=column, stat='percent', common_norm=False, bins=20,
                     color=color, legend=False, ax=ax[0], label='общая совокупность')\
                    .set(ylabel='Частотность/плотность\n (в рамках группы), %')

        ax[0].set_title(column, fontsize=12)
        ax2 = ax[0].twinx()

        if hue:
            sns.kdeplot(data=df, x=column, common_norm=False, hue=hue, fill=True, cut=0,
                        legend=(not i), ax=ax2)

        sns.boxplot(data=df, x=column, legend=False, ax=ax[1])                    .set(xlabel='Значения')
        ax2.set(ylabel='', yticks=[])

        # получаем параметры легенды первого графика и убираем ее
        if hue and not i:
            lines, labels = ax[0].get_legend_handles_labels()
            legend = ax2.get_legend()
            handles = lines + legend.legend_handles
            texts = labels + [hue + ': ' + x.get_text() for x in legend.get_texts()]
            legend.remove()

        i += 1

    # размещаем общую легенду для сетки
    if hue:
        fig.legend(handles, texts, loc='upper center',
                   bbox_to_anchor=(0.5, -0.2), fontsize=12)

    fig.suptitle(title, va='bottom', size=16, y=1.1)
    plt.subplots_adjust(
        left=0,
        right=1,
        top=1,
        bottom=0,
        wspace=0,
        hspace=0
    )
    plt.show()

In [ ]:
# Функция для построения графиков числовых дискретных параметров.

def plot_quant_discrete(df, columns, num_cols, title, hue=None):
    num_rows = -(-len(columns) // num_cols) # округление вверх
    fig = plt.figure(figsize=(num_cols*8, num_rows*5), layout='constrained')

    i = 0

    df_discrete = df.copy()
    df_discrete[columns] = df_discrete[columns].astype(str)

    color = '0.8' if hue else 'C0'

    for column in columns:

        ax = fig.add_subplot(num_rows, num_cols, i+1)
        sns.countplot(data=df_discrete, x=column, stat='percent', legend=False, ax=ax,
                      color=color,  edgecolor='black', order=sorted(df[column].unique()),
                      label='общая совокупность') \
                     .set(ylabel='Частотность (в рамках группы), %', xlabel='Значения', title=column)

        ax.set_title('Признак: '+column, fontsize=12)

        if hue:
            sns.histplot(data=df_discrete, x=column, stat='percent', legend=(not i), discrete=True,
                         common_norm=False, ax=ax, hue=hue, multiple='dodge',
                         shrink=.5)

        if hue and not i:
            lines, labels = ax.get_legend_handles_labels()
            legend = ax.get_legend()
            handles = lines + legend.legend_handles
            texts = labels + [hue + ': ' + x.get_text() for x in legend.get_texts()]
            legend.remove()

        i += 1

    if hue:
        fig.legend(handles, texts, loc='upper center',
                   bbox_to_anchor=(0.5, -0.1), fontsize=12)

    fig.set_constrained_layout_pads(wspace=0.1, hspace=0.1)
    fig.suptitle(title, va='bottom', size=16)
    plt.show()


In [ ]:
# Функция для построения графиков качественных параметров.

def plot_qual(df, columns, num_cols, title, hue=None, top=None):
    num_rows = -(-len(columns) // num_cols) # округление вверх
    fig = plt.figure(figsize=(num_cols*8, num_rows*5), layout='constrained')

    i = 0

    color = '0.8' if hue else 'C0'

    for column in columns:
        ax = fig.add_subplot(num_rows, num_cols, i+1)
        top_names = df[column].value_counts().iloc[:top].index
        data = df[df[column].isin(top_names)]

        sns.countplot(data=data, y=column, stat='percent', legend=False, ax=ax,
                      color=color,  edgecolor='black', order=top_names,
                      label='общая совокупность') \
                     .set(xlabel='Частотность, %', ylabel='Значения', title=column);

        ax.set_title('Признак: '+column, fontsize=12)

        if hue:
            sns.histplot(data=data, y=column, stat='percent', legend=(not i),
                         common_norm=False, ax=ax, hue=hue, multiple='dodge',
                         shrink=.5)

        if hue and not i:
            lines, labels = ax.get_legend_handles_labels()
            legend = ax.get_legend()
            handles = lines + legend.legend_handles
            texts = labels + [hue + ': ' + x.get_text() for x in legend.get_texts()]
            legend.remove()

        i += 1

    if hue:
        fig.legend(handles, texts, loc='upper center',
                   bbox_to_anchor=(0.5, 0), fontsize=12)

    fig.set_constrained_layout_pads(wspace=0.1, hspace=0.1)
    fig.suptitle(title, va='bottom', size=16)
    plt.show()

Построим графики числовых непрерывных параметров.

In [ ]:
num_features_interval = ['funding_total_usd', 'first_funding_at', 'last_funding_at', 'lifetime']

In [ ]:
plot_quant_interval(
    df=startups_train_proc,
    columns=num_features_interval,
    num_cols=4,
    title='Непрерывные признаки в тренировочной выборке',
    hue='status'
)

plot_quant_interval(
    df=startups_test_proc,
    columns=num_features_interval,
    num_cols=4,
    title='Непрерывные признаки в тестовой выборке'
)

Распределения признаков в обучающей и тестовой выборке похожи. Также можно отметить, что в обучающей выборке закрывшиеся стартапы имеют более ранние даты первого и последнего финансирования и меньшие сроки жизни. "Выбросы" по признаку funding_total_usd не будем удалять, т.к. в тестовой выборке тоже есть аналогичные очень высокие значения.

Увеличим масштаб по признаку funding_total_usd.

In [ ]:
threshold = 20000000

plot_quant_interval(
    df=startups_train_proc.query('funding_total_usd < @threshold'),
    columns=['funding_total_usd'],
    num_cols=1,
    title='Непрерывные признаки в тренировочной выборке',
    hue='status'
)

plot_quant_interval(
    df=startups_test_proc.query('funding_total_usd < @threshold'),
    columns=['funding_total_usd'],
    num_cols=1,
    title='Непрерывные признаки в тестовой выборке',
)

Распределения также совпадают. Значительных отличий между закрывшимися и работающими стартапами не наблюдается.

In [ ]:
plot_quant_interval(
    df=startups_train_proc,
    columns=['founded_at', 'closed_at'],
    num_cols=2,
    title='Непрерывные признаки в тренировочной выборке',
)

Теперь можно удалить признаки 'founded_at' и 'closed_at', т.к. они отсутствуют в тестовой выборке.

In [ ]:
startups_train_proc = startups_train_proc.drop(['founded_at', 'closed_at'], axis=1)
startups_train_proc.info()

Построим графики числовых дискретных признаков для тренировочной выборки.



In [ ]:
num_features_discrete = ['funding_rounds']

plot_quant_discrete(
    df=startups_train_proc,
    columns=num_features_discrete,
    num_cols=1,
    title='Дискретные признаки в тренировочной выборке',
    hue='status'
)

plot_quant_discrete(
    df=startups_test_proc,
    columns=num_features_discrete,
    num_cols=1,
    title='Дискретные признаки в тестовой выборке',
)

Частотности признаков схожи. В тренировочной выборке работающие стартапы прошли больше раундов финансирования, чем закрывшиеся (что практически очевидно).

Построим графики категориальных признаков для тренировочной выборки.

In [ ]:
plot_qual(
    df=startups_train_proc,
    columns=cat_features,
    num_cols=5,
    title='Категориальные признаки для тестовой выборки',
    hue='status',
    top=20
)

plot_qual(
    df=startups_test_proc,
    columns=cat_features,
    num_cols=5,
    title='Категориальные признаки для тестовой выборки',
    top=20
)

Частотности признаков также схожи. Большая часть стартапов занимается софтом и биотехнологиями, причем в этих сферах доля работающих превышает долю закрывшихся. Наибольшая доля закрывшихся стартапов - среди тех, где не указана категория, а также в сфере Curated Web. Аналогично, большая часть стартапов - из США, причем здесь так же доля работающих выше, чем доля закрывшихся. Обратная картина - там, где стана не указана, и в России (!).

## Добавление новых признаков

Исходя из имеющегося набора признаков можно добавить признаки:
* признаки, полученные в результате обработки category_list (для уменьшения кардинальности);
* годы первого и последнего финансирования (позволит уменьшить кардинальность и рассматривать эти признаки как дискретные)
* продолжительность финансирования;
* соотношение продолжительности финансирования и продолжительности жизни стартапа;
* интенсивность финансирования (соотношение суммы и продолжительности финансирования).

Последние три признака могут косвенно свидетельствовать об уверенности/неуверенности инвесторов в живучести стартапа. Однако они имеют смысл только в случае, если раундов финансирования было больше 1.

### Обработка category_list

Преобразуем category_list следующим образом:
* разобьем по разделителю "|";
* посчитаем частоту вхождений каждой категории и выделим топ-20;
* для каждой создадим 5 новых столбцов, в которые войдут наиболее частотные категории (либо "иные").

In [ ]:
class TopCategories(BaseEstimator, TransformerMixin):
    def __init__(self, top=20, n_categories=5):
        self.top = top
        self.n_categories = n_categories


    def fit(self, X):
        categories = []

        for lst in X.str.split('|'):
            categories.extend(lst)

        self.top_categories = pd.Series(categories).value_counts()[:self.top]

        return self

    def repl_func(self, x, top_categories, n_categories=2):
            ret = ['Other' for _ in range(n_categories)]
            i = 0

            for index, value in top_categories.items():
                if index in x:
                    ret[i] = index
                    i += 1
                if i == n_categories:
                    break

            return ret

    def transform(self, X):
        df = pd.DataFrame(X.apply(self.repl_func, top_categories=self.top_categories, n_categories=self.n_categories).to_list())
        df.columns = ['category_'+str(i+1) for i in range(self.n_categories)]
        return pd.concat([X, df], axis=1)

    def get_feature_names_out(self):
        pass

In [ ]:
cat_processor = ColumnTransformer(
    [
        ('top_categories', TopCategories(), 'category_list'),
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
startups_train_proc = cat_processor.fit_transform(startups_train_proc)
startups_train_proc.info()

### Показатели, связанные с временны́ми признаками

#### Годы первого и последнего финансирования

In [ ]:
def extract_year(X) -> pd.DataFrame:
    for column in X:
        X[column+'_year'] = X[column].dt.year
    return X

In [ ]:
year_extractor = ColumnTransformer(
    [
        ('year_extractor', FunctionTransformer(extract_year), ['first_funding_at', 'last_funding_at'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
startups_train_proc = year_extractor.fit_transform(startups_train_proc)
startups_train_proc.info()

#### Продолжительность финансирования

In [ ]:
def compute_funding_time(X, name='funding_time') -> pd.DataFrame:
    if X.shape[1] != 2:
        raise Exception('Should be only 2 columns')
    X[name] = abs(pd.Series(X.iloc[:, 1] - X.iloc[:, 0]).dt.days)
    X[name] = X[name].mask(X[name]==0, np.nan)
    return X

In [ ]:
funding_timer = ColumnTransformer(
    [
        ('funding_timer', FunctionTransformer(compute_funding_time), ['first_funding_at', 'last_funding_at'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
startups_train_proc = funding_timer.fit_transform(startups_train_proc)
startups_train_proc.info()

#### Соотношение продолжительности финансирования и продолжительности жизни стартапа

In [ ]:
def compute_funding_lifetime_ratio(X, name='funding_lifetime_ratio') -> pd.DataFrame:
    if X.shape[1] != 2:
        raise Exception('Should be only 2 columns')
    X[name] = pd.Series(X.iloc[:, 0] / X.iloc[:, 1])
    return X

In [ ]:
funding_lifetime_rationer = ColumnTransformer(
    [
        ('funding_lifetime_rationer', FunctionTransformer(compute_funding_lifetime_ratio), ['funding_time', 'lifetime'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
startups_train_proc = funding_lifetime_rationer.fit_transform(startups_train_proc)
startups_train_proc.info()

#### Интенсивность финансирования

In [ ]:
def compute_funding_intensity(X, name='funding_intensity') -> pd.DataFrame:
    if X.shape[1] != 2:
        raise Exception('Should be only 2 columns')
    X[name] = pd.Series(X.iloc[:, 0] / X.iloc[:, 1])
    return X

In [ ]:
funding_intensitier = ColumnTransformer(
    [
        ('funding_intensitier', FunctionTransformer(compute_funding_intensity), ['funding_total_usd', 'funding_time'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
startups_train_proc = funding_intensitier.fit_transform(startups_train_proc)
startups_train_proc.info()

Построим графики для новых признаков

In [ ]:
plot_quant_interval(
    df=startups_train_proc,
    columns=['funding_time', 'funding_lifetime_ratio', 'funding_intensity'],
    num_cols=3,
    title='Новые признаки в тренировочной выборке',
    hue='status'
)

In [ ]:
plot_quant_interval(
    df=startups_train_proc.query('funding_time<2500'),
    columns=['funding_time'],
    num_cols=1,
    title='Новые признаки в тренировочной выборке',
    hue='status'
)

In [ ]:
plot_quant_interval(
    df=startups_train_proc.query('funding_lifetime_ratio<0.8'),
    columns=['funding_lifetime_ratio'],
    num_cols=1,
    title='Новые признаки в тренировочной выборке',
    hue='status'
)

In [ ]:
plot_quant_interval(
    df=startups_train_proc.query('funding_intensity<50000'),
    columns=['funding_intensity'],
    num_cols=1,
    title='Новые признаки в тренировочной выборке',
    hue='status'
)

Относительно заметные различия между закрывшимися и продолжающими работать стартапами видны по признаку funding_lifetime_ratio.

## Корреляционный анализ

Построим матрицу корреляций по методу phik. Не будем рассматривать признаки category_list (слишком большая кардинальность). Признаки с датами преобразуем в чила. Чтобы ускорить расчеты, возьмем случаюную выборку в 25% от датасета.

In [ ]:
df = startups_train_proc.drop(['name', 'category_list'], axis=1).sample(frac=0.25, random_state=RANDOM_STATE)
df['first_funding_at'] = df['first_funding_at'].apply(lambda x: x.timestamp())
df['last_funding_at'] = df['last_funding_at'].apply(lambda x: x.timestamp())

plt.figure(figsize=(10, 10))
sns.heatmap(df.phik_matrix(
                            interval_cols=['funding_total_usd', 'lifetime', 'first_funding_at', 
                                           'funding_rounds', 'last_funding_at', 'funding_time', 
                                           'funding_lifetime_ratio', 'funding_intensity']
                          ),annot=True, fmt='.2f', cmap="coolwarm"
           );
fig, ax = plt.gcf(), plt.gca()
fig.suptitle('Матрица корреляций по методу phik', va='bottom', y=0.9, fontsize=16)
ax.set_xlabel('Признаки')
ax.set_ylabel('Признаки')
cb_ax = fig.axes[1]
cb_ax.set_ylabel("Коэффициент корреляции")
plt.show()

"Географические" параметры сильно коррелированы (что логично). Поскольку наибольший коэффициент корреляции у признака country, оставим только его.

first_funding_at_year и last_funding_at_year, очевидно, сильно коррелированы с first_funding_atr и last_funding_at. Однако "годовые" показатели имеют более высокую корреляцию целевым признаком, поэтому можно оставить только их. first_funding_at_year и last_funding_at_year также коррелированы между собой, что обусловлено большим числом стартапов с 1 раундом финансирования, даты которых совпадают. Учитывая это, оставим оба признака.

Среди пяти категорий, созданных из признака category_list, у первой коэффициент корреляции значительно выше остальных. Оставим только ее.

Также включим funding_rounds и lifetime (по коэффициентам корреляции).


Таким образом, для построения модели отбираем следующие признаки:
* category_1;
* country_code;
* funding_rounds;
* lifetime;
* first_funding_at_year;
* last_funding_at_year;
* funding_lifetime_ratio.

Напишем пайплайн для отбора признаков.

In [ ]:
select_features = [
                 'category_1', 
                 'country_code',
                 'funding_rounds',
                 'lifetime',
                 'first_funding_at_year', 
                 'last_funding_at_year',
                 'funding_lifetime_ratio'
                 ]

In [ ]:
startups_train_proc = startups_train_proc[select_features]
startups_train_proc.info()

## Моделирование

### Формирование итогового пайплайна для обработки данных

In [ ]:
date_features_train = ['first_funding_at', 'last_funding_at', 'founded_at', 'closed_at']
date_features_test = ['first_funding_at', 'last_funding_at']
cat_features = ['category_list', 'country_code', 'state_code', 'region', 'city']

data_preprocessor_for_train = ColumnTransformer(
    [
        ('date_timer', FunctionTransformer(to_date), date_features_train),
        ('cat_imputer', cat_imputer, cat_features),
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

data_preprocessor_for_test = ColumnTransformer(
    [
        ('date_timer', FunctionTransformer(to_date), date_features_test),
        ('cat_imputer', cat_imputer, cat_features),
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

feature_generator = Pipeline(
    [
        ('cat_processor', cat_processor),
        ('year_extractor', year_extractor),
        ('funding_lifetime_rationer', funding_lifetime_rationer)
    ]
)

feature_selector = ColumnTransformer(
    [
        ('selector', FunctionTransformer(None), select_features)
    ],
    remainder='drop',
    verbose_feature_names_out=False
).set_output(transform='pandas')


data_pipe_for_train = Pipeline(
    [
        ('data_preprocessor', data_preprocessor_for_train),
        ('lifetimer', lifetimer),
        ('funding_timer', funding_timer), 
        ('feature_generator', feature_generator),
        ('funding_intensitier', funding_intensitier),
        ('feature_selector', feature_selector)
    ]
)

data_pipe_for_test = Pipeline(
    [
        ('data_preprocessor', data_preprocessor_for_test),
        ('funding_timer', funding_timer), 
        ('feature_generator', feature_generator),
        ('funding_intensitier', funding_intensitier),
        ('feature_selector', feature_selector)
    ]
)

### Построение модели CatBoost

In [ ]:
cat = CatBoostClassifier(
    eval_metric='F1',
    random_seed=RANDOM_STATE,
    verbose=200
)

In [ ]:
X = startups_train_proc
y = startups_train['status']
y = (y == 'closed')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y)

cat_features_boost = list(X.select_dtypes(include='object').columns)
pool_train = Pool(X_train, y_train, cat_features=cat_features_boost)
pool_test = Pool(X_test, y_test, cat_features=cat_features_boost)

In [ ]:
cat.fit(pool_train, eval_set=pool_test, verbose=200)

In [ ]:
print(f'Метрика f1 для лучшей модели на тренировочных данных: {f1_score(y_train, cat.predict(pool_train)):.4f}')
print(f'Метрика f1 для лучшей модели на валидационных данных: {f1_score(y_test, cat.predict(pool_test)):.4f}')

Модель "из коробки" показала результат 0.8718 на валидационных данных. Неплохо, но можно улучшить.

### Работа с признаками

Попробуем различные комбинации признаков на модели "из коробки". Сначала построим модель для всех признаков.

In [ ]:
def update_pipe(select_features):
    feature_selector = ColumnTransformer(
        [
            ('selector', FunctionTransformer(None), select_features)
        ],
        remainder='drop',
        verbose_feature_names_out=False
    ).set_output(transform='pandas')
    data_pipe_for_train.steps[-1] = ('feature_selector', feature_selector)
    data_pipe_for_test.steps[-1] = ('feature_selector', feature_selector)

def prepare_data(select_features):
    X = startups_train.drop('status', axis=1)
    y = startups_train['status']
    y = (y == 'closed')
    
    update_pipe(select_features=select_features)
    
    X = data_pipe_for_train.fit_transform(X)
    
    cat_features_boost = list(X.select_dtypes(include='object').columns)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y)
    
    pool_train = Pool(X_train, y_train, cat_features=cat_features_boost)
    pool_test = Pool(X_test, y_test, cat_features=cat_features_boost)

    return pool_train, pool_test, X_test

def train_model(select_features):
    pool_train, pool_test, X_test = prepare_data(select_features)

    cat = CatBoostClassifier(
        eval_metric='F1',
        random_seed=RANDOM_STATE,
        verbose=200
    )
    
    cat.fit(pool_train, eval_set=pool_test, verbose=200)

    print(f'Метрика f1 для лучшей модели на тренировочных данных: {f1_score(y_train, cat.predict(pool_train)):.4f}')
    print(f'Метрика f1 для лучшей модели на валидационных данных: {f1_score(y_test, cat.predict(pool_test)):.4f}')
    return cat, X_test, pool_test

In [ ]:
select_features = [
                 'category_list',
                 'category_1', 'category_2', 'category_3', 'category_4', 'category_5',
                 'funding_total_usd',
                 'country_code', 'state_code', 'region', 'city',
                 'funding_rounds',
                 'first_funding_at', 'last_funding_at',
                 'lifetime',
                 'first_funding_at_year', 'last_funding_at_year',
                 'funding_time',
                 'funding_lifetime_ratio',
                 'funding_intensity'
                 ]

cat, X_test, pool_test = train_model(select_features)

Лучше, чем на исходной модели, несмотря на то что ряд признаков коррелирован.

Рассмотрим вклад признаков. 

In [ ]:
pd.DataFrame(
        {'feature_names': X_test.columns,
         'feature_importance': cat.get_feature_importance(pool_test)
        }
    ).sort_values(by='feature_importance', ascending=False)

In [ ]:
pool_train, pool_test, X_test = prepare_data(select_features)

cat = CatBoostClassifier(
    eval_metric='F1',
    random_seed=RANDOM_STATE,
    verbose=200
)
    
sf = cat.select_features(pool_train, eval_set=pool_test, features_for_select=select_features, num_features_to_select=8)
sf['selected_features_names']

In [ ]:
pd.DataFrame(
        {'feature_names': X_test.columns,
         'feature_importance': cat.get_feature_importance(pool_test)
        }
    ).sort_values(by='feature_importance', ascending=False)

Уберем из списка признаков также category_1, поскольку он является производным от category_list, у которого выше важность.

### Подбор гиперпараметов с использованием Optuna

Для подбора гиперпараметров используем optuna.
Будем перебирать следующие основные параметры (исходя ихз документации CatBoost):
* bootstrap_type;
* depth;
* l2_leaf_reg;
* learning_rate;
* loss_function;
* random_strength.

ПРИМЕЧАНИЕ: поиск параметров занимает длительное время, поэтому здесь для первой попытки уже приведены параметры, подобранные офлайн.

In [ ]:
select_features = [
                 'category_list',
                 'funding_total_usd',
                 'country_code',
                 'first_funding_at', 'last_funding_at',
                 'lifetime',
                 'funding_lifetime_ratio'
                 ]

pool_train, pool_test, X_test = prepare_data(select_features)

In [ ]:
def objective(trial):
    params = {
              'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
              'depth': trial.suggest_int('depth', 8, 11),
              'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3, 5),
              'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.1),
              'loss_function': trial.suggest_categorical('loss_function', ['CrossEntropy', 'Logloss']),
              'random_strength': trial.suggest_float('random_strength', 0.4, 1.0)
             }

    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 3)
    else:
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1)

    model = CatBoostClassifier(
        **params,
        random_seed=RANDOM_STATE,
        boosting_type='Ordered',
        eval_metric='F1',
    )
    
    pruning_callback = CatBoostPruningCallback(trial, 'F1')
    model.fit(
        pool_train,
        eval_set=pool_test,
        verbose=200,
        callbacks=[pruning_callback]
    )
    pruning_callback.check_pruned()
    trial.set_user_attr(key='best_cat', value=model)
    y_pred = model.predict(pool_test)
    return f1_score(y_test, y_pred)

def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key='best_cat', value=trial.user_attrs['best_cat'])

sampler = TPESampler(seed = RANDOM_STATE)
study = optuna.create_study(
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5, n_min_trials=500),
                            direction='maximize', sampler=sampler
                           )

# Ранее подобранные параметры
study.enqueue_trial(
    {
        'bootstrap_type': 'Bayesian',
        'depth': 9,
        'l2_leaf_reg': 3.3279204553296013,
        'learning_rate': 0.07364361140839092,
        'loss_function': 'CrossEntropy',
        'random_strength': 0.9272666496043003,
        'bagging_temperature': 0.581964049763424
    }
)

study.optimize(objective, n_trials=1, callbacks=[callback])
cat = study.user_attrs['best_cat']

print('Лучшее значение целевой метрики:', study.best_value)
print('Параметры модели:', study.best_params)

### Прогноз на тестовых данных

Построим финальный пайплайн с обученной моделью.

In [ ]:
# Набор признаков должен совпадать в тренировочном и тестовом датасетах
df = startups_train.drop(['status', 'founded_at', 'closed_at'], axis=1)
df['lifetime'] = 0
df = data_pipe_for_test.fit_transform(df)
df.info()

In [ ]:
pipe_final= Pipeline(
    [
        ('data_pipe_for_test', data_pipe_for_test),
        ('model', cat)
    ]
)

In [ ]:
X = startups_test
predict = pd.Series(pipe_final.predict(X))
predict = predict.apply(lambda x: 'closed' if x else 'operating')
df = pd.concat([startups_test['name'], predict], axis=1)
df.columns = ['name', 'status']
df = df.set_index('name')
df.head()

## Анализ важности признаков

Построим график shap.

In [ ]:
# pool_train, pool_test, X_test = prepare_data(select_features)
# shap рассматривает признаки datetime как категориальные, поэтому преобразуем их в числовые.
X_test['first_funding_at'] = X_test['first_funding_at'].apply(lambda x: x.timestamp())
X_test['last_funding_at'] = X_test['last_funding_at'].apply(lambda x: x.timestamp())

explainer = shap.TreeExplainer(cat)
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values, show=False)

fig, ax = plt.gcf(), plt.gca()
fig.suptitle('Влияние факторов на целевой признак по методу Шепли', fontsize=16)
ax.set_xlabel('Значения SHAP (влияние на целевой признак)')
ax.set_ylabel('Признаки')
cb_ax = fig.axes[1]
cb_ax.set_ylabel("Значение признака")
plt.show()

Мы видим следующее влияние:
* lifetime - чем он больше, тем меньше вероятность, что стартап закроется;
* first_funding_at, last_funding_at - чем они выше (позже), тем больше вероятность, что стартап закроется (при том что в исследовательском анализе мы видели на графиках обратную картину; возможно, это обусловлено взаимовлиянием признаков);
* funding_total_usd - однозначное влияние не установлено, но высокие значения признака несколько повышают вероятность закрытия;
* funding_lifetime_ratio - однозначное влияние не установлено, но для низкого соотношения меньше вероятность закрытия.

По категориальным признакам числовое влияние не установлено, но наш исследовательский анализ показывает:
* меньше вероятность закрытия для стартапов в категориях Software и Biotechnology, больше - в категории Curated Web;
* в США вероятность закрытия меньше, в России - больше.

## Общий вывод

Мы провели исследовательский анализ данных о стартапах с целью определения факторов, влияющих на их успешность, и построили ML-модель для прогнозирования продолжения их деятельности.

Основные результаты исследовательского анализа:
* Закрывшиеся стартапы имеют более ранние даты первого и последнего финансирования, меньшее число раундов финансирования и меньшие сроки жизни.
* Работающие стартапы прошли больше раундов финансирования, чем закрывшиеся.
* В сфере производства софта и биотехнологий доля работающих стартапов выше, наибольшая доля закрывшихся стартапов в сфере Curated Web;
* В США доля работающих стартапов выше, чем доля закрывшихся. Обратная картина - в России.
* У закрывшихся стартапов выше отношение времени финансирования ко времени жизни (т.е. работающие стартапы больше времени живут без финансирования).

Основные результаты моделирования:
* С помощью CatBoost получена ML-модель предсказания закрытия стартапов, которая на валидационной выборке показала метрику F1 0.8834.
* Анализ важности признаков подтвердил выводы, полученные на этапе исследовательского анализа, за исключением признаков first_funding_at, last_funding_at - здесь ситуация обратная (чем позже финансирование, тем больше вероятность закрытия).
* Отдельные факторы (число раундов финансирования, штат, регион и город) не оказывают значимого влияния на вероятность закрытия, однако объем финансирования имеет прямую связь (пусть и не сильную) с вероятностью закрытия.

Таким образом, будущим стартаперам можно рекомендовать:
* выбирать перспективные направления работы (например, софт и биотехнологии);
* выбирать более благоприятные для стартапов страны;
* не полагаться только на внешнее финансирование.

## P.S.

В ходе работы с моделями и подбора признаков было экспериментально установлены, что лучшие метрики получаются на следующем наборе признаков:
* category_list;
* funding_total_usd;
* country_code;
* state_code;
* region;
* city;
* funding_rounds;
* first_funding_at;
* last_funding_at;
* lifetime;
* first_funding_at_year;
* last_funding_at_year.

C помощью Optuna были подбраны гиперпараметры модели, которые показали наилучший результат (модель сохранена в файле).

In [ ]:
select_features = [
                 'category_list',
                 'funding_total_usd',
                 'country_code',
                 'state_code', 'region', 'city',
                 'funding_rounds',
                 'first_funding_at', 'last_funding_at',
                 'lifetime',
                 'first_funding_at_year', 'last_funding_at_year'
                 ]

pool_train, pool_test, X_test = prepare_data(select_features)

try:
    cat.load_model('/kaggle/input/best-catboost-model/other/default/1/final.model')
except:
    cat.load_model('final.model')

print(f'Метрика f1 для лучшей модели на тренировочных данных: {f1_score(y_train, cat.predict(pool_train)):.4f}')
print(f'Метрика f1 для лучшей модели на тестовых данных: {f1_score(y_test, cat.predict(pool_test)):.4f}')

Построим график shap.

In [ ]:
X_test['first_funding_at'] = X_test['first_funding_at'].apply(lambda x: x.timestamp())
X_test['last_funding_at'] = X_test['last_funding_at'].apply(lambda x: x.timestamp())

explainer = shap.TreeExplainer(cat)
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values, show=False)

fig, ax = plt.gcf(), plt.gca()
fig.suptitle('Влияние факторов на целевой признак по методу Шепли', fontsize=16)
ax.set_xlabel('Значения SHAP (влияние на целевой признак)')
ax.set_ylabel('Признаки')
cb_ax = fig.axes[1]
cb_ax.set_ylabel("Значение признака")
plt.show()

На графике видно разнонаправленное влияние first_funding_at и first_funding_at_year, а также last_funding_at и last_funding_at_year. Влияние "годовых" показателей более соответствует тому, что мы видели в исследовательском анализе. Возможно, именно поэтому модель с этими признаками имеет более высокие метрики.